Шаг 1: Импорт

In [6]:
import pandas as pd
import requests
from io import StringIO
import numpy as np
import yfinance as yf
from datetime import datetime

print("Шаг 1: Библиотеки импортированы.")

Шаг 1: Библиотеки импортированы.


Шаг 2: Загрузка данных (Объединенная и улучшенная версия)

In [7]:
# --- Этот блок объединяет Часть 2 и Часть 3 ---

try:
    # --- Загрузка списка IPO ---
    url_2024 = "https://stockanalysis.com/ipos/2024/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/91.0.4472.124 Safari/537.36'
        )
    }
    
    response = requests.get(url_2024, headers=headers, timeout=10)
    tables = pd.read_html(StringIO(response.text))
    ipos_2024_df = tables[0]

    # --- Фильтрация и получение списка тикеров ---
    ipos_2024_df['IPO Date'] = pd.to_datetime(ipos_2024_df['IPO Date'])
    filtered_ipos = ipos_2024_df[ipos_2024_df['IPO Date'] < datetime(2024, 6, 1)]
    ipo_tickers = filtered_ipos['Symbol'].tolist()

    # --- Проверка №1: Убедимся, что список тикеров не пуст ---
    if not ipo_tickers:
        raise ValueError("Список тикеров для загрузки пуст!")
    
    print(f"Шаг 2.1: Список IPO получен. Найдено {len(ipo_tickers)} тикеров.")

    # --- Загрузка котировок с yfinance ---
    print(f"Шаг 2.2: Загружаем исторические данные для {len(ipo_tickers)} тикеров...")
    stocks_df = yf.download(ipo_tickers, start='2024-01-01', end='2025-06-10')

    # --- Проверка №2: Убедимся, что DataFrame не пуст ---
    if stocks_df.empty:
        raise ValueError("yfinance не смог загрузить данные. DataFrame пуст.")

    # --- Преобразование данных ---
    stocks_df = stocks_df.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index()
    print(f"Шаг 2.3: Данные для {stocks_df['Ticker'].nunique()} тикеров успешно загружены и обработаны.")
    
except Exception as e:
    print(f"На шаге 2 произошла ошибка: {e}")
    # Создаем пустой DataFrame, чтобы следующий шаг не выдавал NameError
    stocks_df = pd.DataFrame()

Шаг 2.1: Список IPO получен. Найдено 77 тикеров.
Шаг 2.2: Загружаем исторические данные для 77 тикеров...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  77 of 77 completed


Шаг 2.3: Данные для 77 тикеров успешно загружены и обработаны.


/tmp/ipykernel_41352/3649832375.py:38: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stocks_df = stocks_df.stack(level=1).rename_axis(['Date', 'Ticker']).reset_index()


Шаг 3: Расчеты и финальный ответ

In [8]:
# Убедимся, что предыдущий шаг сработал
if 'stocks_df' in locals() and not stocks_df.empty:
    print("Шаг 3: Выполняем расчеты...")

    # Сортируем для корректных вычислений
    stocks_df = stocks_df.sort_values(by=['Ticker', 'Date'])

    # Расчет годового роста (252 торговых дня)
    stocks_df['growth_252d'] = stocks_df.groupby('Ticker')['Close'].transform(lambda x: x / x.shift(252))

    # Расчет волатильности
    stocks_df['volatility'] = stocks_df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(30).std() * np.sqrt(252))

    # Расчет коэффициента Шарпа
    risk_free_rate = 0.045
    stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 1 - risk_free_rate) / stocks_df['volatility']

    # Фильтруем данные на нужную дату
    final_data = stocks_df[stocks_df['Date'] == '2025-06-06'].copy()

    # Находим медианное значение
    median_sharpe = final_data['Sharpe'].median()

    print("\nОписательная статистика для проверки:")
    print(final_data[['growth_252d', 'Sharpe']].dropna().describe().round(4))

    print("\n" + "="*50)
    print("                ОТВЕТ НА ВОПРОС 2")
    print("="*50)
    print(f"Медианный коэффициент Шарпа для IPO 2024 года: {median_sharpe:.4f}")
    print("="*50)
else:
    print("Переменная 'stocks_df' не была создана. Пожалуйста, убедитесь, что Шаг 2 выполнился без ошибок.")

Шаг 3: Выполняем расчеты...

Описательная статистика для проверки:
Price  growth_252d   Sharpe
count      73.0000  73.0000
mean        1.2279  -0.2336
std         1.4802   0.6971
min         0.0250  -4.0575
25%         0.2935  -0.2576
50%         0.7632  -0.0141
75%         1.4467   0.0131
max         8.0974   0.4959

                ОТВЕТ НА ВОПРОС 2
Медианный коэффициент Шарпа для IPO 2024 года: -0.0141


In [10]:
# Убедимся, что DataFrame first_day_returns_df существует
if 'first_day_returns_df' in locals() and not first_day_returns_df.empty:
    
    # --- Шаг 1: Задаем предложенные варианты ответа ---
    possible_months = [10, 7, 4, 2]
    print(f"Предложенные варианты для проверки (в месяцах): {possible_months}")

    # --- Шаг 2: Формируем список колонок для анализа ---
    # Названия колонок, которые соответствуют предложенным вариантам
    columns_to_check = [f'future_growth_{m}m' for m in possible_months]
    
    # --- Шаг 3: Рассчитываем среднюю доходность только для этих колонок ---
    # Используем .loc для выбора нужных колонок и затем считаем среднее
    average_growth_for_options = first_day_returns_df[columns_to_check].mean()

    print("\nСредняя доходность для каждого из предложенных вариантов:")
    print(average_growth_for_options.round(4))
    
    # --- Шаг 4: Находим лучший вариант из предложенных ---
    # Находим колонку с максимальным средним значением
    best_option_col = average_growth_for_options.idxmax()
    
    # Извлекаем количество месяцев из названия колонки
    best_months = int(best_option_col.replace('future_growth_', '').replace('m', ''))

    # --- Финальный вывод ---
    print("\n" + "="*60)
    print("      ВЫБОР ОПТИМАЛЬНОГО ОТВЕТА ИЗ ПРЕДЛОЖЕННЫХ ВАРИАНТОВ")
    print("="*60)
    print(f"Среди вариантов {possible_months} месяцев, наилучшую среднюю доходность")
    print(f"показывает период в {best_months} месяца (доходность = {average_growth_for_options.max():.4f}).")
    print(f"\n=> Программный ответ: {best_months}")
    print("="*60)

else:
    print("Переменная 'first_day_returns_df' не была создана. Пожалуйста, выполните код для Вопроса 3.")

Предложенные варианты для проверки (в месяцах): [10, 7, 4, 2]

Средняя доходность для каждого из предложенных вариантов:
future_growth_10m    0.9133
future_growth_7m     0.8463
future_growth_4m     0.8227
future_growth_2m     0.9369
dtype: float64

      ВЫБОР ОПТИМАЛЬНОГО ОТВЕТА ИЗ ПРЕДЛОЖЕННЫХ ВАРИАНТОВ
Среди вариантов [10, 7, 4, 2] месяцев, наилучшую среднюю доходность
показывает период в 2 месяца (доходность = 0.9369).

=> Программный ответ: 2
